In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# yolov3.weights, yolov3-tiny.weights 파일 다운로드
from urllib.request import urlretrieve
urlretrieve('https://pjreddie.com/media/files/yolov3.weights','yolov3.weights')
urlretrieve('https://pjreddie.com/media/files/yolov3-tiny.weights', 'yolov3-tiny.weights')

In [ ]:
# yolov3.weights, yolov3-tiny.weights 파일 drive로 옮겨놓기
!mv yolov3.weights /content/gdrive/MyDrive/CV/Object\ Detection_YOLO
print('yolov3.weights move complete!!')

!mv yolov3-tiny.weights /content/gdrive/MyDrive/CV/Object\ Detection_YOLO
print('yolov3-tiny.weights move complete!!')

In [ ]:
# dataset 다운로드
!git clone https://github.com/pjreddie/darknet.git

In [ ]:
# 필요한 파일만 복사
!cp darknet/cfg/yolov3.cfg /content/gdrive/MyDrive/CV/Object\ Detection_YOLO
print('yolov3.cfg copy complete!!')

!cp darknet/cfg/yolov3-tiny.cfg /content/gdrive/MyDrive/CV/Object\ Detection_YOLO
print('yolov3-tiny.cfg copy complete!!')

!cp darknet/data/coco.names /content/gdrive/MyDrive/CV/Object\ Detection_YOLO
print('coco.names copy complete!!')

In [ ]:
# 다운 받았던 파일 제거
!rm -r darknet

In [ ]:
# 모듈 임포트
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

In [ ]:
# OpenCV 버전 확인
print('OpenCV version:', cv2.__version__)

In [ ]:
# detection으로 인정할 최소 신뢰도 지정
min_confidence = 0.5

In [ ]:
# YOLO 시작
net = cv2.dnn.readNet('/content/gdrive/MyDrive/CV/Object Detection_YOLO/yolov3.weights',
                      '/content/gdrive/MyDrive/CV/Object Detection_YOLO/yolov3.cfg')

classes = []    # detection 할 Object(Class) list 배열 정의

with open('/content/gdrive/MyDrive/CV/Object Detection_YOLO/coco.names', 'r') as f:
    # 80개의 Object(class)를 구분할 수 있는 Object의 이름을 classes 배열에 넣는다.
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Object 마다 컬러를 하나씩 다르게 지정
colors = np.random.uniform(0, 255, size = (len(classes), 3))

In [ ]:
# 원본 이미지 로드
img = cv2.imread('/content/gdrive/MyDrive/CV/Object Detection_YOLO/image/image/Street.jpg')
img = cv2.resize(img, None, fx=0.7, fy=0.7)
height, width, channels = img.shape

In [ ]:
print('# # # # Original Image # # # #')
cv2_imshow(img)

In [ ]:
# Yolo3 Detecting model
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

net.setInput(blob)
outs = net.forward(output_layers)   # outs에는 detection 된 Object가 있음

class_ids = []          # detection 한 Class id 저장하는 배열
confidences = []        # detection 한 Class의 신뢰도를 저장하는 배열
boxes = []              # detection 한 boxing 정보를 저장하는 배열

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)    # detection 한 Class id
        confidence = scores[class_id]   # detection 한 Class의 신뢰도
        if confidence > min_confidence:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])              # boxing 정보를 boxes 배열에 저장
            confidences.append(float(confidence))   # 신뢰도를 confidences 배열에 저장
            class_ids.append(class_id)              # Class id 를 class_ids 배열에 저장

# 박스안에 박스(노이즈)를 하나로 만들어준다.
indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)

font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:    # 노이즈가 제거된 박스만 표시
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])      # Class 이름
        print(i, label)

        color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
        cv2.rectangle(img, (x, y - 20), (x + w, y), color, -1)
        cv2.putText(img, label, (x + 5, y - 5), font, 1, (255, 255, 255), 1)

cv2_imshow(img)

In [ ]:
# 원본 이미지 로드
img = cv2.imread('/content/gdrive/MyDrive/CV/Object Detection_YOLO/image/image/Street.jpg')
img = cv2.resize(img, None, fx=0.7, fy=0.7)
height, width, channels = img.shape

# Yolo3 Detecting model
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

net.setInput(blob)
outs = net.forward(output_layers)   # outs에는 detection 된 Object가 있음

class_ids = []          # detection 한 Class id 저장하는 배열
confidences = []        # detection 한 Class의 신뢰도를 저장하는 배열
boxes = []              # detection 한 boxing 정보를 저장하는 배열

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)    # detection 한 Class id
        confidence = scores[class_id]   # detection 한 Class의 신뢰도
        if confidence > min_confidence and class_id == 'car':
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])              # boxing 정보를 boxes 배열에 저장
            confidences.append(float(confidence))   # 신뢰도를 confidences 배열에 저장
            class_ids.append(class_id)              # Class id 를 class_ids 배열에 저장

# 박스안에 박스(노이즈)를 하나로 만들어준다.
indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)

font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:    # 노이즈가 제거된 박스만 표시
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])      # Class 이름
        if label == 'car':                      # Class 가 car인 것만 화면에 표시
            print(i, label)

            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
            cv2.rectangle(img, (x, y - 20), (x + w, y), color, -1)
            cv2.putText(img, label, (x + 5, y - 5), font, 1, (255, 255, 255), 1)

cv2_imshow(img)